# T4. model 的搭建与 driver 的概念

&emsp; 1 &ensp; fastNLP 中预训练模型的使用
 
&emsp; &emsp; 1.1 &ensp; 

&emsp; &emsp; 1.2 &ensp; 

&emsp; 2 &ensp; fastNLP 中使用 Pytorch 搭建模型

&emsp; &emsp; 2.1 &ensp; 

&emsp; &emsp; 2.2 &ensp; 

&emsp; 3 &ensp; fastNLP 中的 driver

&emsp; &emsp; 3.1 &ensp; 

&emsp; &emsp; 3.2 &ensp; 